In [11]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
# SANITIZAÇÃO DE DADOS DEPOIS DE ANÁLISES PELO DATASPELL
data_file_name = '../dataset_customer_churn.csv'
print(f"Reading data from {data_file_name}")
data = pd.read_csv(data_file_name, sep='^')
is_NAN = data[data.isna().any(axis=1)]
print("Removing low importance features...")
# Certas colunas dos dados foram removidas por sua baixa importância nas análises de features, podendo assim criar
# um modelo muito mais otimizado sem sacrificar sua eficiência.
data.drop(labels=['A006_REGISTRO_ANS', 'CODIGO_BENEFICIARIO', 'CLIENTE', 'CD_USUARIO', 'CODIGO_FORMA_PGTO_MENSALIDADE',
                  'A006_NM_PLANO', 'CD_ASSOCIADO', 'ESTADO_CIVIL'], axis=1,
          inplace=True)
print("Removing NAN's and outliers from DataFrame")
data.drop(is_NAN.index, axis=0, inplace=True)
# outlier extremo com + de 500 anos de plano ativo
data.drop(182212, axis=0, inplace=True)
print("Replacing categoric features with only 2 options with 0 and 1")
dict_replace = {
    "SIM": 1,
    "NAO": 0,
    'F': 0,
    'M': 1,
    'DESATIVADO': 1,
    'ATIVO': 0,

}
data.replace(dict_replace, inplace=True)
data.drop(data[data['QTDE_DIAS_ATIVO'] == 1790].index, axis=0, inplace=True)

# REDUÇÃO DE CARDINALIDADE NA TABELA 'QTDE_DIAS_ATIVO'
QTDE_DIAS_ATIVO_MENOR_QUE_365 = np.array(data['QTDE_DIAS_ATIVO'] < 365)
data['QTDE_DIAS_ATIVO_MENOR_QUE_365'] = 0
data.loc[QTDE_DIAS_ATIVO_MENOR_QUE_365, 'QTDE_DIAS_ATIVO_MENOR_QUE_365'] = 1
QTDE_DIAS_ATIVO_MENOR_QUE_1000 = np.array((data['QTDE_DIAS_ATIVO'] >= 365) & (data['QTDE_DIAS_ATIVO'] < 1000))
data['QTDE_DIAS_ATIVO_MENOR_QUE_1000'] = 0
data.loc[QTDE_DIAS_ATIVO_MENOR_QUE_1000, 'QTDE_DIAS_ATIVO_MENOR_QUE_1000'] = 1
QTDE_DIAS_ATIVO_MAIOR_QUE_1000 = np.array(data['QTDE_DIAS_ATIVO'] >= 1000)
data['QTDE_DIAS_ATIVO_MAIOR_QUE_1000'] = 0
data.loc[QTDE_DIAS_ATIVO_MAIOR_QUE_1000, 'QTDE_DIAS_ATIVO_MAIOR_QUE_1000'] = 1
data.drop(labels='QTDE_DIAS_ATIVO', inplace=True, axis=1)

data_dummified = pd.get_dummies(data)

x = data_dummified


Reading data from ../dataset_customer_churn.csv
Removing low importance features...
Removing NAN's and outliers from DataFrame
Replacing categoric features with only 2 options with 0 and 1


In [13]:
from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()
data_scaled = stdscaler.fit_transform(data_dummified)
n_clusters_choice = int(input("How much clusters do you want? (5 is the standard): "))
print("Analizing and creating Clusters...")
kmeans = KMeans(n_clusters=n_clusters_choice, n_init=10, max_iter=300, random_state=1010)
kmeans.fit(data_scaled)
data_dummified['CLUSTER'] = kmeans.labels_
description = data_dummified.groupby('CLUSTER')
n_clients = description.size()
description = description.mean()
description['n_clients'] = n_clients
print('Clusters overall descriptions: ')
print(description)
y_pred = kmeans.fit_predict(data_dummified)

Analizing and creating Clusters...
Clusters overall descriptions: 
         NUM_BENEFICIARIOS_FAMILIA  SITUACAO      IDADE      SEXO  \
CLUSTER                                                             
0                         1.882799  0.961717  42.430797  0.344394   
1                         2.160086  0.878679  45.110311  0.367692   
2                         1.910954  0.108915  37.296028  0.331576   
3                         1.852084  0.753588  37.388917  0.417338   
4                         2.023702  0.277607  37.530932  0.383938   

         QTDE_ATENDIMENTOS  QTDE_ATO_COBERTO_EXECUTADO  \
CLUSTER                                                  
0                 8.802841                   31.661106   
1                 2.395054                    9.814964   
2                 1.465313                    4.477167   
3                 0.231569                    0.720901   
4                 2.831660                    8.515131   

         QTDE_ATO_N_COBERTO_EXECUTADO  REA

In [14]:
from sklearn import metrics
labels = kmeans.labels_
labels = labels.reshape(-1, 1)
y_pred = y_pred.reshape(-1, 1)
silhouette = metrics.silhouette_score(y_pred, labels, metric='euclidean')
print(silhouette)

C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1.0


In [ ]:
data_dummified["cluster"] = labels

In [ ]:
data_dummified.groupby("cluster").describe()

In [ ]:
description = data_dummified.groupby("cluster")
n_clients = description.size()
description = description.mean()
description['n_clients'] = n_clients
description